In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

driver = webdriver.Chrome()

driver.get("https://seinet.com.mk/search")
time.sleep(5)

dropdown = Select(driver.find_element(By.ID, 'formIssuerId'))
issuers = []

for option in dropdown.options:
    issuer_id = option.get_attribute('value')
    issuer_name = option.text.strip()
    if issuer_id:
        issuers.append((issuer_id, issuer_name))

data = []

for issuer_id, issuer_name in issuers:
    if issuer_id == "0":
        continue

    url = f"https://seinet.com.mk/search/{issuer_id}"
    print(f"\nПреземање податоци за издавач: {issuer_name} ({url})")

    driver.get(url)
    time.sleep(5)
    
    page_num = 1

    while True:
        try:
            css_selector = "#root > main > div.mt-3.container > div > div > div > div > table > tbody > tr"
            rows = driver.find_elements(By.CSS_SELECTOR, css_selector)

            for index in range(len(rows)):
                try:
                    rows = driver.find_elements(By.CSS_SELECTOR, css_selector)
                    row = rows[index]

                    element_news = row.find_element(By.CSS_SELECTOR, f"td:nth-child(2)")
                    news_title = element_news.text.strip()

                    driver.execute_script("arguments[0].click();", element_news)
                    time.sleep(5)

                    news_html = driver.page_source
                    news_soup = BeautifulSoup(news_html, 'html.parser')
                    news_description = news_soup.find('div', class_='text-left ml-auto mr-auto col-md-11')

                    if news_description:
                        paragraphs = news_description.find_all('p')
                        full_description = ' '.join(paragraph.get_text(strip=True) for paragraph in paragraphs)
                    else:
                        full_description = "Нема достапен опис."

                    data.append({
                        "Компанија": issuer_name,
                        "Наслов": news_title,
                        "Опис": full_description
                    })

                    driver.back()
                    time.sleep(5)

                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))

                    if page_num > 1:
                        pagination_link = f'#root > main > div.mt-3.container > div > div > div > ul > li:nth-child({page_num}) > a'
                        driver.find_element(By.CSS_SELECTOR, pagination_link).click()
                        time.sleep(5)

                    rows = driver.find_elements(By.CSS_SELECTOR, css_selector)

                except Exception as e:
                    print(f"Грешка при обработка на редот {index + 1}: {e}")

            try:
                active_page = driver.find_element(By.CSS_SELECTOR, '#root > main > div.mt-3.container > div > div > div > ul > li.active.page-item > a')

                next_page = active_page.find_element(By.XPATH, '../following-sibling::li[not(contains(@class, "disabled"))]/a')

                WebDriverWait(driver, 10).until(EC.element_to_be_clickable(next_page))
                try:
                    cookie_banner = driver.find_element(By.CLASS_NAME, 'cookie-consent-banner')
                    if cookie_banner:
                        close_button = cookie_banner.find_element(By.TAG_NAME, 'button')
                        close_button.click()
                        time.sleep(2)
                except:
                    print("Банерот за колачиња не е пронајден или веќе е затворен")

                next_page.click()
                time.sleep(5)
                page_num += 1
            except:
                break

        except Exception as e:
            break

driver.quit()

df = pd.DataFrame(data)
csv_path = 'C:/Users/User/Desktop/companies_with_news.csv'
df.to_csv(csv_path, index=False, encoding='utf-8-sig')
